# Readme

Pre-requisites
*   Set up weight and bias account: [Wandb](https://wandb.ai/site/)
*   Upload dataset on drive following the hierarchy below: <br>
<pre>
MyDrive
|_ Final_project
   |_ 2000dataset.zip
</pre>

*   Upload the generator and the zip containing the source code in drive following the herachy below:
<pre>
MyDrive
|_ Final_project
   |_ HAT
      |_ HAT.zip
      |_Real_HAT_GAN_SRx4.pth
</pre>

*   for **inference** using generator obtained after fine tuning on the dataset, ensure net_g_5000.pth is at the hierarchy below:

 <pre>
MyDrive
|_ Final_project
   |_ HAT
      |_ net_g_5000.pth
</pre>


**Fine-Tuning**: Follow steps 1 to 4 in order. <br> <br>
**Inference**:
*   The HAT code and the generator is required for inference.
*   Run step 1, step 2 and Step 5. *Note: At step 5.2, replace the dataroot_lq with correct LR image path to run inference on and prertain_network_g with correct generator path.*

#1 Install dependencies and login to wandb

In [ ]:
!pip install wandb -qqq
!pip install basicsr==1.3.4.9 -qqq
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 24.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files, drive
import os
import wandb
import shutil

In [ ]:
#mount drive to colab
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Corrects import line from degradations.py as this version is obsolete
#https://github.com/xinntao/Real-ESRGAN/issues/801
#Note: if python version was updated on colab, change the file_path to correct one.

file_path = "/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py"
line_to_replace = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
new_line = "from torchvision.transforms.functional import rgb_to_grayscale"

with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [new_line if line.strip() == line_to_replace else line for line in lines]

with open(file_path, 'w') as file:
    file.writelines(lines)

print(f"Replacing line '{line_to_replace}' with '{new_line}' in file {file_path}")

Replacing line 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' with 'from torchvision.transforms.functional import rgb_to_grayscale' in file /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py


In [ ]:
# Install and log in to Weights & Biases; not required if doing inference only
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dsha43925 (dsha43925-middlesex-university-mauritius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# 2 Get code

In [ ]:
# get code from drive and set up environment for HAT
code_zip_path = '/content/drive/MyDrive/Final_project/HAT/HAT.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path
%cd /content/HAT

!pip install -r requirements.txt
!python setup.py develop

total 12
drwx------ 5 root root 4096 Sep 23 19:40 drive
drwxrwxrwx 7 root root 4096 Sep  4 06:15 HAT
drwxr-xr-x 1 root root 4096 Sep 19 13:40 sample_data
/content/HAT
/usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.12/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
      

#3 Get dataset from drive

In [ ]:
dataset_zip_path = '/content/drive/MyDrive/Final_project/2000dataset.zip'
!unzip -q $dataset_zip_path -d $unzip_path

In [ ]:
rm "/content/dataset/train/lr/desktop.ini"

#4 Get pre-trained model from drive

In [ ]:
!cp "/content/drive/MyDrive/Final_project/HAT/Real_HAT_GAN_SRx4.pth" \
   "/content/HAT/experiments/pretrained_models/Real_HAT_GAN_SRx4.pth"

In [ ]:
model_path = "/content/HAT/experiments/pretrained_models/Real_HAT_GAN_SRx4.pth"

# Check file exists and size
if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"Model found! Size: {size_mb:.1f} MB")
    # Expected size: 81.2 MB for medium and 158 MB for Large
else:
    print("Error: File not copied correctly!")

Model found! Size: 162.4 MB


#5 Fine-tune pre-trained model on satellite images

## 5.1 Create configuration file for training

In [ ]:
# Define the YAML content
yaml_content = """# general settings
name: train_Real_HAT_GAN_SRx4_finetune_from_mse_model
model_type: RealHATGANModel
scale: 4
num_gpu: auto
manual_seed: 100

# USM the ground-truth
l1_gt_usm: True
percep_gt_usm: True
gan_gt_usm: False
high_order_degradation: False

# dataset and data loader settings
datasets:
  train:
    name: sat_images
    type: PairedImageDataset
    dataroot_gt: /content/dataset/train/hr
    dataroot_lq: /content/dataset/train/lr
    meta_info: /content/dataset/meta_info/meta_info_satelliteimages_train.txt
    io_backend:
      type: disk

    gt_size: 256
    use_hflip: true
    use_rot: true

    # data loader
    use_shuffle: true
    num_worker_per_gpu: 1
    batch_size_per_gpu: 2
    dataset_enlarge_ratio: 2
    prefetch_mode: ~

  val_1:
    name: val_sat_images
    type: PairedImageDataset
    dataroot_gt: /content/dataset/val/hr
    dataroot_lq: /content/dataset/val/lr
    meta_info: /content/dataset/meta_info/meta_info_satelliteimages_val.txt
    io_backend:
      type: disk

# network structures
network_g:
  type: HAT
  upscale: 4
  in_chans: 3
  img_size: 64
  window_size: 16
  compress_ratio: 3
  squeeze_factor: 30
  conv_scale: 0.01
  overlap_ratio: 0.5
  img_range: 1.
  depths: [6, 6, 6, 6, 6, 6]
  embed_dim: 180
  num_heads: [6, 6, 6, 6, 6, 6]
  mlp_ratio: 2
  upsampler: 'pixelshuffle'
  resi_connection: '1conv'

network_d:
  type: UNetDiscriminatorSN
  num_in_ch: 3
  num_feat: 64
  skip_connection: True

# path
path:
  pretrain_network_g: /content/HAT/experiments/pretrained_models/Real_HAT_GAN_SRx4.pth
  param_key_g: 'params_ema'
  strict_load_g: true
  resume_state: ~

# training settings
train:
  ema_decay: 0.999
  optim_g:
    type: Adam
    lr: !!float 2e-5
    weight_decay: 0
    betas: [0.9, 0.99]
  optim_d:
    type: Adam
    lr: !!float 2e-5
    weight_decay: 0
    betas: [0.9, 0.99]

  scheduler:
    type: MultiStepLR
    milestones: [2000, 3000, 4000]
    gamma: 0.5

  total_iter: 5000
  warmup_iter: -1  # no warm up

  # losses
  pixel_opt:
    type: L1Loss
    loss_weight: 1.0
    reduction: mean
  perceptual_opt:
    type: PerceptualLoss
    layer_weights:
      # before relu
      'conv1_2': 0.1
      'conv2_2': 0.1
      'conv3_4': 1
      'conv4_4': 1
      'conv5_4': 1
    vgg_type: vgg19
    use_input_norm: true
    perceptual_weight: !!float 1.0
    style_weight: 0
    range_norm: false
    criterion: l1
  # gan loss
  gan_opt:
    type: GANLoss
    gan_type: vanilla
    real_label_val: 1.0
    fake_label_val: 0.0
    loss_weight: !!float 1e-1

  net_d_iters: 1
  net_d_init_iters: 0


# validation settings
val:
  val_freq: 100
  save_img: true
  pbar: False

  metrics:
    psnr:
      type: calculate_psnr
      crop_border: 4
      test_y_channel: true
      better: higher
    ssim:
      type: calculate_ssim
      crop_border: 4
      test_y_channel: true
      better: higher
    niqe:
      type: calculate_niqe
      crop_border: 4
      input_order: 'HWC'
      convert_to: 'y'

# logging settings
logger:
  print_freq: 100
  save_checkpoint_freq: 500
  use_tb_logger: true
  wandb:
    project: HAT-satellite-SR
    resume_id: ~

# dist training settings
dist_params:
  backend: nccl
  port: 29500
"""

# Create the directory if it doesn't exist
!mkdir -p /content/HAT/options/train/

# Write to the YAML file
with open('/content/HAT/options/train/train_HAT-GAN_SRx4_finetune.yml', 'w') as f:
    f.write(yaml_content)

# Verify the file was created
!ls -l /content/HAT/options/train/

total 4
-rw-r--r-- 1 root root 3189 Sep 18 15:20 train_HAT-GAN_SRx4_finetune.yml


## 5.2 Fine tune model

In [ ]:
!CUDA_VISIBLE_DEVICES=0 torchrun --nproc_per_node=1 --master_port=4321 hat/train.py -opt /content/HAT/options/train/train_HAT-GAN_SRx4_finetune.yml --launcher none

Disable distributed.
2025-09-18 15:21:04,749 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.3.4.9
	PyTorch: 2.8.0+cu126
	TorchVision: 0.23.0+cu126
2025-09-18 15:21:04,749 INFO: 
  name: train_Real_HAT_GAN_SRx4_finetune_from_mse_model
  model_type: RealHATGANModel
  scale: 4
  num_gpu: 1
  manual_seed: 100
  l1_gt_usm: True
  percep_gt_usm: True
  gan_gt_usm: False
  high_order_degradation: False
  

#6 Inference

## 6.1 Get zipfile containing low resolution image to test

In [ ]:
#Upload the zip file containing low resolution images for inference.
#code_zip_path = '/content/drive/MyDrive/Final_project/testImages.zip'
code_zip_path = '/content/drive/MyDrive/Final_project/LRInferenceImages.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

total 16
drwx------ 5 root root 4096 Sep 23 19:40 drive
drwxrwxrwx 8 root root 4096 Sep 23 19:41 HAT
drwxrwxrwx 2 root root 4096 Sep 23 18:42 LRInferenceImages
drwxr-xr-x 1 root root 4096 Sep 19 13:40 sample_data


## 6.2 Create configuration file for inference

replace with correct path in config:
pretrain_network_g: '/content/HAT/experiments/pretrained_models/Real_HAT_GAN_SRx4.pth'

In [ ]:
# Define the YAML content as a string
test_config_yml = """
name: HAT-L_SRx4_ImageNet-pretrain
model_type: HATModel
scale: 4
num_gpu: 1
manual_seed: 100

tile:
  tile_size: 64
  tile_pad: 32

datasets:
  test_1:
    name: customSatelliteIMages
    type: SingleImageDataset
    dataroot_lq: /content/LRInferenceImages
    io_backend:
      type: disk

# network structures
network_g:
  type: HAT
  upscale: 4
  in_chans: 3
  img_size: 64
  window_size: 16
  compress_ratio: 3
  squeeze_factor: 30
  conv_scale: 0.01
  overlap_ratio: 0.5
  img_range: 1.
  depths: [6, 6, 6, 6, 6, 6]
  embed_dim: 180
  num_heads: [6, 6, 6, 6, 6, 6]
  mlp_ratio: 2
  upsampler: 'pixelshuffle'
  resi_connection: '1conv'

# path
path:
  pretrain_network_g: '/content/drive/MyDrive/Final_project/HAT/net_g_2000.pth'
  strict_load_g: true
  param_key_g: 'params_ema'

# validation settings
val:
  save_img: true
  suffix: ~  # add suffix to saved images, if None, use exp name
"""

output_path = "/content/HAT/options/test/HAT-L_inference.yml"

# Create directories not exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save to yml
with open(output_path, "w") as f:
    f.write(test_config_yml)

print(f"YAML config saved to {output_path}")


YAML config saved to /content/HAT/options/test/HAT-L_inference.yml


## 5.3 Run for inference

In [ ]:
!python hat/test.py -opt options/test/HAT-L_inference.yml

Disable distributed.
2025-09-23 19:43:29,869 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.3.4.9
	PyTorch: 2.8.0+cu126
	TorchVision: 0.23.0+cu126
2025-09-23 19:43:29,870 INFO: 
  name: HAT-L_SRx4_ImageNet-pretrain
  model_type: HATModel
  scale: 4
  num_gpu: 1
  manual_seed: 100
  tile:[
    tile_size: 64
    tile_pad: 32
  ]
  datasets:[
    test_1:[
      name: customSatelliteIMages
      type: 

In [ ]:
#download the results
zip_path = "/content/HAT200G.zip"
folder_to_zip = "/content/HAT/results/HAT-L_SRx4_ImageNet-pretrain/visualization/customSatelliteIMages"
shutil.make_archive(zip_path.replace(".zip",""), 'zip', folder_to_zip)
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>